In [5]:
import pandas as pd
import numpy as np
import random
import difflib

# loading training data
train_df = pd.read_csv('./data/training.txt', sep="\t", header = None)
train_df.columns  = [1, 2, 3, 4, 5, 6, 7, "class"]

train_df = train_df.applymap(lambda x: True if x == 2 else False)
        
#loading testing data
test_df = pd.read_csv('./data/test.txt', sep="\t", header = None)
test_df.columns = [1, 2, 3, 4, 5, 6, 7, "class"]

test_df = test_df.applymap(lambda x: True if x == 2 else False)

In [19]:
#Method returning a decision tree implementing the importance function 
def decisionTreeLearning(examples, attributes, parent_examples):
    attributes = attributes
    if len(examples) == 0:
        return pluralityValue(parent_examples)
    elif allSameClass(examples):
        return getClass(examples)
    elif len(attributes) == 0:
        return pluralityValue(examples)
    else:
        parent_examples = examples
        next_att = importance(examples, attributes)
        next_attributes = attributes
        next_attributes.remove(next_att)
        tree = [next_att, {True: decisionTreeLearning(examples[examples[next_att] == True], next_attributes, examples), False: decisionTreeLearning(examples[examples[next_att] == False], next_attributes, parent_examples)} ]
        return tree

#Method returning a dumb decision tree implementing without importance function 
def dumbDecisionTreeLearning(examples, attributes, parent_examples):
    if len(examples) == 0:
        return pluralityValue(parent_examples)
    elif allSameClass(examples):
        return getClass(examples)
    elif len(attributes) == 0:
        return pluralityValue(examples)
    else:
        parent_examples = examples
        next_att = random.sample(set(attributes), 1)[0]
        next_attributes = attributes
        next_attributes.remove(next_att)
        tree = [next_att, {True: dumbDecisionTreeLearning(examples[examples[next_att] == True], next_attributes, examples), False: dumbDecisionTreeLearning(examples[examples[next_att] == False], next_attributes, parent_examples)} ]
        return tree
    
#Method returning plurality value
def pluralityValue(examples):
    true_count = (examples["class"]).sum()
    false_count = len(examples) - true_count
    if true_count > false_count:
        return True
    elif true_count < false_count:
        return False
    else:
        return random.sample(set([True, False]), 1)[0]

#Method returning true if all remaining examples are of same class
def allSameClass(examples):
    return len(examples["class"].value_counts()) == 1

#Method returning the most common class    
def getClass(examples):
    return examples["class"].iloc[0]

#Method implementing the gain function
def gain(examples, attribute):
    pos = getPosCount(examples)
    return B(pos/float(len(examples))) - remainder(examples, attribute)
    
#Method implementing the B function 
def B(q):
    if q <= 0 or q == 1 :
        return 0
    result = - (q*np.log2(q) + (1 - q)*np.log2(1 - q))
    return result

#Method returning the remainder function
def remainder(examples, attribute):
    class_pos_count = getPosCount(examples)
        
    att_pos_split = examples[examples[attribute] == True]
    att_neg_split =  examples[examples[attribute] == False]
    
    att_pos_class_pos_count = getPosCount(att_pos_split)
    att_neg_class_pos_count = getPosCount(att_neg_split)
        
    remainder_att_pos = 0 if len(att_pos_split) == 0 else (len(att_pos_split)/float(len(examples))) * B(att_pos_class_pos_count/float(len(att_pos_split))) 
    remainder_att_neg = 0 if len(att_neg_split) == 0 else (len(att_neg_split)/float(len(examples))) * B(att_neg_class_pos_count/float(len(att_neg_split)))
    return remainder_att_pos + remainder_att_neg  
 
#Method that returns the number of instances classified as true
def getPosCount(examples):
    return (examples["class"]).sum()

#Method that implements the importance function
def importance(examples, attributes):
    gains_dicc = {att: gain(examples, att) for att in attributes}
    max_gain = max(gains_dicc.values())
    important_atts = filter(lambda att: gains_dicc[att] >= max_gain, attributes)
    return random.sample(set(important_atts), 1)[0]

#Helper method to classify an item/row acording to tree
def classify(item, tree):
    while True:
        att = tree[0]
        value = item[att]
        dicc = tree[1]
        tree = dicc[value]
        if not isinstance(tree, list):
            return tree

#Helper method to classify a range of items
def getResults(tree, df):
    results = []
    for index, row in df.iterrows():
        results.append(classify(row, tree))
    return results

#Helper method asses performance, by comparing results obtained with tree with the actual classification
def assessPerformance(results, actual_results):
    sm=difflib.SequenceMatcher(None,results,actual_results)
    return sm.ratio()

   


In [24]:
#Lets try to build dumb decision tree (no information improvement)
dumb_tree = dumbDecisionTreeLearning(train_df, [1, 2, 3, 4, 5, 6, 7], train_df)
print dumb_tree

#lets see how the dumb tree performs by averaging the performances of 100 runs
dumbTreePerformances = []
for i in range(100):
    dumb_tree = dumbDecisionTreeLearning(train_df, [1, 2, 3, 4, 5, 6, 7 ], train_df)
    results = getResults(dumb_tree, test_df)
    actual_results = test_df["class"].tolist()
    dumbTreePerformances.append(assessPerformance(results, actual_results))

print (sum(dumbTreePerformances)/len(dumbTreePerformances))





[7, {False: False, True: [2, {False: False, True: [5, {False: False, True: [6, {False: False, True: [3, {False: [4, {False: False, True: [1, {False: False, True: True}]}], True: False}]}]}]}]}]
0.613928571429


In [27]:
#Lets try to build a smart decision tree (with information improvement)
tree = decisionTreeLearning(train_df, [1, 2, 3, 4, 5, 6, 7], train_df)
print tree

#lets see how the tree performs by averaging the performances of 100 runs
treePerformances = []
for i in range(100):
    tree = decisionTreeLearning(train_df, [1, 2, 3, 4, 5, 6, 7], train_df)
    results = getResults(tree, test_df)
    actual_results = test_df["class"].tolist()
    treePerformances.append(assessPerformance(results, actual_results))

print (sum(treePerformances)/len(treePerformances))

[1, {False: False, True: [5, {False: False, True: [6, {False: False, True: [7, {False: False, True: [4, {False: [2, {False: [3, {False: False, True: True}], True: True}], True: True}]}]}]}]}]
0.693571428571
